In [1]:
import random
import hashlib
import os
from datetime import datetime, timedelta
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.exceptions import InvalidSignature

part A and B

In [2]:
def generate_bank_private_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    pem_private = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ).decode()

    return pem_private

def generate_bank_public_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()

    pem_public = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ).decode()

    return pem_public

def sign_serial_number(blinded_serial_number, private_key):
    # Convert blinded serial number to a hex string and then to an integer
    m = int(blinded_serial_number.to_bytes((blinded_serial_number.bit_length() + 7) // 8, byteorder='big').hex(), 16)

    # Compute the hash of the blinded serial number
    h = hashlib.sha256(m.to_bytes((m.bit_length() + 7) // 8, byteorder='big')).digest()

    # Sign the hash using the private key
    signature = private_key.sign(
        h,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    return signature


def blind(serial_number, public_key):
    # Convert serial number to a hex string and then to an integer
    m = int(serial_number.encode().hex(), 16)

    # Choose a random blinding factor
    r = random.randint(1, public_key.public_numbers().n - 1)

    # Compute the blinded serial number
    blinded_m = (m * pow(r, public_key.public_numbers().e, public_key.public_numbers().n)) % public_key.public_numbers().n

    return blinded_m, r

def generate_coin_with_expiration(private_key_pem, value, expiration_days):
    # Deserialize the private key from PEM format
    private_key = serialization.load_pem_private_key(
        private_key_pem.encode(),
        password=None
    )

    # Generate a unique serial number for the coin
    serial_number = "COIN-" + str(datetime.now().timestamp())

    # Blind the serial number
    blinded_serial_number, blinding_factor = blind(serial_number, private_key.public_key())

    # Calculate the expiration date
    expiration_date = datetime.now() + timedelta(days=expiration_days)

    # Create the coin object with value, expiration date, and blinding factor
    coin = {
        "value": value,
        "expiration_date": expiration_date.strftime("%Y-%m-%d"),
        "serial_number": serial_number,
        "blinded_serial_number": blinded_serial_number,
        "blinding_factor": blinding_factor,
        "public_key": private_key.public_key(),
        "private_key": private_key,
        "signed_serial_number": sign_serial_number(blinded_serial_number, private_key)
    }

    return coin

def store_blinded_serial_numbers(blinded_serial_numbers):
    with open("ecoins-Spec.txt", "w") as file:
        for coin in blinded_serial_numbers:
            file.write(f"Serial Number: {coin['serial_number']}\n")
            file.write(f"Blinded Serial Number: {coin['blinded_serial_number']}\n")
            file.write(f"Blinding Factor: {coin['blinding_factor']}\n")
            file.write(f"Value: {coin['value']}\n\n")

def unblind_and_sign_serial_numbers(coins):
    unblinded_signed_serial_numbers = []
    for coin in coins:
        unblinded_serial_number = (coin['blinded_serial_number'] * pow(coin['blinding_factor'], -1, coin['public_key'].public_numbers().n)) % coin['public_key'].public_numbers().n
        unblinded_signed_serial_numbers.append((unblinded_serial_number.to_bytes((unblinded_serial_number.bit_length() + 7) // 8, byteorder='big'), coin['signed_serial_number']))
    return unblinded_signed_serial_numbers

def store_unblinded_signed_serial_numbers(unblinded_signed_serial_numbers):
    with open("ecoins.txt", "w") as file:
        for unblinded_serial_number, signature in unblinded_signed_serial_numbers:
            file.write(f"Unblinded Serial Number: {unblinded_serial_number.hex()}\n")
            file.write(f"Bank Signature: {signature.hex()}\n\n")

In [4]:
private_key_5 = generate_bank_private_keys()
public_key_5 = generate_bank_public_keys()

private_key_10 = generate_bank_private_keys()
public_key_10 = generate_bank_public_keys()

private_key_20 = generate_bank_private_keys()
public_key_20 = generate_bank_public_keys()

# Generate coins with different denominations and expiration dates, each using a different key pair
coin1 = generate_coin_with_expiration(private_key_5, 5, 50)
coin2 = generate_coin_with_expiration(private_key_10, 10, 60)
coin3 = generate_coin_with_expiration(private_key_20, 20, 100)

# Store the blinded serial numbers, blinding factors, and signatures of the coins in a file
coins = [coin1, coin2, coin3]
store_blinded_serial_numbers(coins)

# Unblind and sign the serial numbers of the coins using the corresponding private keys
unblinded_signed_serial_numbers = unblind_and_sign_serial_numbers(coins)

# Store the unblinded signed serial numbers in a file
store_unblinded_signed_serial_numbers(unblinded_signed_serial_numbers)

In [5]:
coins[0]

{'value': 5,
 'expiration_date': '2023-09-10',
 'serial_number': 'COIN-1689986934.607556',
 'blinded_serial_number': 18504569935426620980851993758289009419323964994339338698169633449121242043632744615993430263933113629902575054488871335904826850086350405846996463097988181558898377975581725240564617246104580787822421810567348501646358294358193815305718951996798195184902503961377847688465351648523442397311155820025274305004111669846731208674568850511958751926281780575245433920605539334910978857240202717852585692254393487642998905570446508206943277386667662478894013732126139808983478871837337532600939264446975287668627313652483570735513644989960800311172890863332738680748458091228039341649253458052389385150251654287841145084391,
 'blinding_factor': 2478254182323670789353937653500470915020462573901694805503797045303393332264349545362286847448509937997562166968159438170155662624405592598566929244007427840901864943035456443949575297191160285595712111216258156439752722656221474592890816911387

Part C

In [6]:
# Generate bank's public key
def generate_bank_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()

    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_pem, public_pem
# Generate bank's public-private keys
private_key_pem, public_key_pem = generate_bank_keys()

# Deserialize the public key from PEM format
public_key = serialization.load_pem_public_key(public_key_pem)


def validate_coin(coin, public_key, spent_serial_numbers):
    # Retrieve coin information
    serial_number = coin["serial_number"]
    blinded_serial_number = coin["blinded_serial_number"]
    value = coin["value"]
    expiration_date = datetime.strptime(coin["expiration_date"], "%Y-%m-%d")
    #signature = coin["signature"]


    # Verify expiration date
    current_date = datetime.now()
    if current_date > expiration_date:
        print("Coin has expired.")
        return False

    # Check serial number in spent serial numbers database
    if serial_number in spent_serial_numbers:
        print("Coin has already been spent.")
        return False

    # Add serial number to spent serial numbers database
    spent_serial_numbers.add(serial_number)


    print("Coin is valid and can be accepted.")
    return True

# Example usage:
received_coin = {
    "serial_number": "COIN-1234567897665",
    "blinded_serial_number": "COIN-1234567890.123",
    "value": 10,
    "expiration_date": "2023-1-1",

}

received_coin1 = {
    "serial_number": "COIN-1234567890",
    "blinded_serial_number": "COIN-1234567890.123",
    "value": 10,
    "expiration_date": "2023-12-31",

}
received_coin2 = {
    "serial_number": "COIN-1234567890",
    "blinded_serial_number": "COIN-1234567896.123",
    "value": 10,
    "expiration_date": "2023-12-31",

}
# Assuming you have the public_key instance

# Set up the database of spent serial numbers

spent_serial_numbers = set()


#valid = validate_coin(coin_to_validate, public_key, spent_serial_numbers )

# Validate the received coin
valid = validate_coin(received_coin1, public_key, spent_serial_numbers)

#print(spent_serial_numbers)

if valid:
    # Process the coin and accept the payment
    print("Payment accepted.")
else:
    # Reject the coin and inform the merchant
    print("Payment rejected.")

Coin is valid and can be accepted.
Payment accepted.


In [7]:
#Same serial number ->> double spending scenario
valid = validate_coin(received_coin2, public_key, spent_serial_numbers)
if valid:
    # Process the coin and accept the payment
    print("Payment accepted.")
else:
    # Reject the coin and inform the merchant
    print("Payment rejected.")

Coin has already been spent.
Payment rejected.


In [8]:
# expairation date has come.
valid = validate_coin(received_coin, public_key, spent_serial_numbers)
if valid:
    # Process the coin and accept the payment
    print("Payment accepted.")
else:
    # Reject the coin and inform the merchant
    print("Payment rejected.")

Coin has expired.
Payment rejected.


In [9]:
coins[2]

{'value': 20,
 'expiration_date': '2023-10-30',
 'serial_number': 'COIN-1689986934.716949',
 'blinded_serial_number': 23528635997810512073580627957928641506449396182392218376829436739041505375924151169634341992858748416398396731590676766743019244442917842023805930954356690300836005004799129776862888132600952061399850032547913759773352445133963173208039581969073589294608318527188227699249666245228612069805820635386272604893025330726636908785075014091019291903213179019640347763492360328413686719151030954711403827762844881062594733799581319695373199216325071802384827701735111239537779533505464941549953333406062587571259068928414103845430283384941738571253481893409522029532283585833922193432869853203153707333445597339866899415058,
 'blinding_factor': 559818231615553396690465346443560264992929573148076839655001019125062662552368700492783689851963587627508354214197636751035374638882377213782854901853911936532996134834962265418613581190228390127723264730621083388746017633283537764625582124978

In [10]:
# third coin of the customer
valid = validate_coin(coins[2], public_key, spent_serial_numbers)
if valid:
    # Process the coin and accept the payment
    print("Payment accepted.")
else:
    # Reject the coin and inform the merchant
    print("Payment rejected.")

Coin is valid and can be accepted.
Payment accepted.
